In [1]:
#ライブラリ読み込み
import requests
import json
import os
from WSMethods import uploadPredefinedProfile, uploadProfile, splitWspString
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
import pyvisa

スペアナ部

In [13]:
rm = pyvisa.ResourceManager()

speana_address = "-------"

with rm.open_resource(speana_address) as instr:
    print(instr.query("*IDN?"))

In [ ]:
def getOSA(address, trace_name):
    c = 299792458

    #光スペアナのx軸（波長）取得

    rm = pyvisa.ResourceManager()
    
    with rm.open_resource(address) as instr:
        wavelength = instr.query(':TRACe:X? '+trace_name)
        wavelength = wavelength.split(',')
        wavelength = [float(ii) for ii in wavelength]
        wavelength = np.array(wavelength)#m

    freq = [c/ii/10**12 for ii in wavelength]#THz


    #光スペアナのy軸（dBm/nm）取得
    with rm.open_resource(address) as instr:
        nrf = instr.query(':CALCULATE:MATH:TRF:PNBW:BAND?')#正規化帯域幅
    #print(nrf)#0.1という数値が出てくるものの何のため？


    with rm.open_resource(address) as instr:
        data = instr.query(':TRACe:Y:PDENsity? '+trace_name+',1nm')
        data = data.split(',')
        data = [float(ii) for ii in data]
        data = np.array(data)

    optFreq = freq
    optSpectrum = data

    optFreq = optFreq[::-1]
    optSpectrum = optSpectrum[::-1]

    optFreq = np.hstack((s-0.05,optFreq))
    optFreq = np.hstack((optFreq,e+0.05))
    optSpectrum = np.hstack((optSpectrum[0],optSpectrum))
    optSpectrum = np.hstack((optSpectrum,optSpectrum[-1]))
    
    return optFreq,optSpectrum

In [ ]:
optFreq, optSpectrum = getOSA(speana_address, 'TRA') #たぶんdB単位で出てくるみたい
plt.plot(optFreq, optSpectrum)


